# Introduction

This notebook is an exercise from Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow, by Aurélien Geron.

It aims to build different Deep Neural Networks to predict images in the dataset CIFAR-10, availiable in https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [ ]:
import matplotlib.pyplot as plt

# see random image
some_index = np.random.choice(X_train.shape[0])
some_image = X_train[some_index]
some_label = y_train[some_index]
plt.imshow(some_image)
print(some_label)

In [ ]:
# unique values in y_train
y_train_unique = np.unique(y_train)
y_train_unique

In [ ]:
# plot one image from each class in a subplot
fig = plt.figure(figsize=(16, 8))
for label in y_train_unique:
    image_index = np.random.choice(np.where(y_train == label)[0])
    plt.subplot(1, len(y_train_unique), label + 1)
    plt.imshow(X_train[image_index])
    plt.axis('off')
    plt.title(label)

### a)
Build a DNN with 20 hidden layers of 100 neurons each. Use the He initialization and the ELU activation function.

In [ ]:
X_train.shape

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(42)

In [ ]:
model = keras.models.Sequential()
for i in range(21):
    if i == 0:
        model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
    else:
        model.add(keras.layers.Dense(100, kernel_initializer='he_normal', activation='elu'))
model.add(keras.layers.Dense(10, activation='softmax'))

## b)
Using Nadam Optimization and early stopping, train the network.

In [ ]:
# Using Nadam Optimization and early stopping, train the network.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(),
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# subplots of loss and accuracy
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], name='Training Loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'], name='Validation Loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['accuracy'], name='Training Accuracy'), row=1, col=2)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_accuracy'], name='Validation Accuracy'), row=1, col=2)
fig.update_layout(height=600, width=1000)
fig.show()